# Pridobivanje podatkov o objavah iz COBISS

In [1]:
import os
import time
from datetime import datetime
import gzip
import mechanicalsoup
import urllib

Pridobimo podatke za obdobje tekočega in prejšnjega leta. 

In [2]:
now = datetime.now()
print('začetek: {:}'.format(now.strftime('%Y-%m-%d %H:%M:%S')))

tstamp = now.strftime('%Y-%m-%d-%H%M%S')
print('Čas poizvedbe: {:}'.format(tstamp))

doleta = now.year
odleta = doleta - 1

začetek: 2019-07-09 22:40:04
Čas poizvedbe: 2019-07-09-224004


Podatke shranimo v mapo `cobiss`.

In [3]:
target_folder = 'downloads/cobiss-org'
if not os.path.isdir(target_folder):
    os.makedirs(target_folder)
    # če mapa ne obstaja, potem moramo pridobiti podatke od začetka organizacije
    odleta = 1990

Letnici pretvorimo v niza.

In [4]:
doleta = str(doleta)
odleta = str(odleta)
print('Obdobje: {:}-{:}'.format(odleta, doleta))

Obdobje: 2018-2019


Podatki o UL FRI, šifra organizacije `1539`.

In [5]:
org_id='1539'

url_start='http://www.sicris.si/search/org_biblio.aspx?lang=slv&mstid={:}'.format(org_id)
#url_step2='http://splet02.izum.si/cobiss/bibliography?lang=slv'
#url_xml='http://splet02.izum.si/cobiss/bibpersonal/ConnectForm.jsp?lang=slv&mstid={:}&odleta={:}&doleta={:}&wos=on&biblunit=1&izpis=11&veda=T&points=connect&altmetrics=0&units=1&DaljeC=DALJE'.format(org_id, odleta, doleta)
#url_html='http://splet02.izum.si/cobiss/bibpersonal/ConnectForm.jsp?lang=slv&mstid={:}&odleta={:}&doleta={:}&wos=on&biblunit=1&izpis=7&veda=T&points=connect&altmetrics=0&units=1&DaljeC=DALJE'.format(org_id, odleta, doleta)
print('start: {:}'.format(url_start))

start: http://www.sicris.si/search/org_biblio.aspx?lang=slv&mstid=1539


V ime datoteke zapišemo čas poizvedbe.

In [6]:
target_file = '{:}/{:}_cobiss'.format(target_folder, tstamp)

In [7]:
def get_url(izpis):
    browser = mechanicalsoup.StatefulBrowser()
    browser.open(url_start)
    browser.select_form()
    # browser.get_current_form().print_summary()
    browser.submit_selected()
    browser.select_form()
    browser['odleta'] = odleta
    browser['doleta'] = doleta
    browser['izpis'] = izpis
    # browser.get_current_form().print_summary()
    response = browser.submit_selected()
    url = response.text.split('URL=')[1].split('">')[0]
    return url

Podamo zahtevo za generiranje datoteke html.

In [8]:
url_html = get_url('7')
print('URL datoteke html: {:}'.format(url_html))

URL datoteke html: http://izumbib.izum.si/bibliografije/Z20190709224008-1539.html


Podamo zahtevo za generiranje datoteke xml.

In [9]:
url_xml = get_url('11')
print('URL datoteke xml: {:}'.format(url_xml))

URL datoteke xml: http://izumbib.izum.si/bibliografije/Z20190709224009-1539.xml


Preverjaj, dokler datoteki nista generirani.

In [10]:
ponovi = True
while ponovi:
    print('{:}: preverjam, ali je izpis pripravljen...'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    with urllib.request.urlopen(url_html) as response:
        text_html = response.read()

    with urllib.request.urlopen(url_xml) as response:
        text_xml = response.read()
        
    ponovi = '<info>Prosimo, po' in text_xml.decode() or '<meta http-equiv="REFRESH"' in text_html.decode()
    if ponovi:
        print('čakam...')
        time.sleep(120)
print('Pripravljeni datoteki sta prenešeni iz strežnika.')

2019-07-09 22:40:09: preverjam, ali je izpis pripravljen...
čakam...
2019-07-09 22:42:09: preverjam, ali je izpis pripravljen...
čakam...
2019-07-09 22:44:10: preverjam, ali je izpis pripravljen...
Pripravljeni datoteki sta prenešeni iz strežnika.


In [11]:
assert text_xml.decode().endswith('</Bibliography>\n')
assert text_html.decode().endswith('</html>\n')

Shranimo datoteko tipa html in xml.

In [12]:
with open(target_file + '.html', 'wb') as fout:
    fout.write(text_html)

with gzip.open(target_file + '.xml.gz', 'wb') as fout:
    fout.write(text_xml)

In [13]:
print('konec: {:}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

konec: 2019-07-09 22:44:28
